In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter, FFMpegWriter
import flopy
import shutil, os
from IPython.display import HTML


In [ ]:
def hmean(x1,x2):
    return(2/np.sum((1/x1,1/x2)))

In [ ]:
dx = 10.
dy = 10.
nx = 40
b = 1.
kx = 1
h0_x0 = 10.
h0_xL = 5
R = 0.001
x = np.linspace(0,nx,nx)*dx
k = kx*np.ones_like(x)
Rx = R*dx*dy
k[np.floor(nx/3).astype(int):] = 20
G = Rx
tol = 1e-6

# fixed head both boundaries

### first implicit solution as reference

In [ ]:
x = np.linspace(0,nx,nx)*dx
k = kx*np.ones_like(x)
Rx = R*dx*dy
h = np.zeros_like(x)
# add some heterogeneity
k[np.floor(nx/3).astype(int):] = 20
G = Rx

LHS = np.zeros((nx,nx))
RHS = np.ones(nx) * Rx
for i in range(1,nx-1): 
    D = - hmean(k[i-1],k[i]) * b * dy / (dx)
    F = - hmean(k[i+1],k[i]) * b * dy / (dx)
    E = -D - F
    LHS[i,i-1] = D
    LHS[i,i]=E
    LHS[i,i+1]= F
    RHS[i] = G

LHS[0,0] = 1
LHS[nx-1,nx-1] = 1

RHS[nx-1] = h0_xL
RHS[0]= h0_x0

In [ ]:
h_imp=np.linalg.solve(LHS,RHS)

### now explicit inefficient solution for fixed_head case

In [ ]:
# initialize the heads 
h_old = np.mean((h0_x0,h0_xL)) * np.ones_like(x)
h_old[0] = h0_x0
h_old[nx-1] = h0_xL

In [ ]:
conv=[np.inf]
h = np.atleast_2d(h_old.copy())
for j in range(10000):
    if conv[-1]>tol:
        h_curr = h[-1].copy()
        for i in range(1, len(x)-1):
            D = - hmean(k[i-1],k[i]) * b * dy / (dx)
            F = - hmean(k[i+1],k[i]) * b * dy / (dx)
            E = -D - F
            h_curr[i] = (G-D*h_curr[i-1]-F*h_curr[i+1])/E
        h = np.vstack((h,h_curr))

        conv.append(np.max(np.abs(h[-1,:]-h_old)))
        h_old=h[-1,:].copy()

In [ ]:
h[1,:]-h[0,:]

### make a schmancy animation of iterating through the explicit solution

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,4))
ax[0].plot(x,h_imp, 'o-', label='"Truth"')
l0, = ax[0].plot([],[], label='Current head solution')
l1, = ax[1].plot([],[], lw=2)
line=[l0,l1]
ax[0].set_xlim((0,x.max()))
ax[1].set_xlim((0,x.max()))
ax[0].set_ylim((0,12))
ax[1].set_ylim((-.2,.2))
plt.suptitle('Fixed Head Boundary')
def init():
    ax[0].set_ylim((0,12))
    ax[1].set_ylim((-.2,.2))
    ax[0].set_title('')
    ax[1].set_title('')


def update(i):
    if i==0:
        h_old = h[i,:]
    else:
        h_old = h[i-1,:]
    line[0].set_data(x, h[i,:])
    ax[0].legend()
    line[1].set_data(x, h[i,:]-h_old)
    ax[0].set_title(f'Head Solution: iter {i}')
    ax[1].set_title(f'Error = {conv[i]:.4e}')
    return line
ani = FuncAnimation(fig, update, range(h.shape[0]), init_func=init)
writer = FFMpegWriter(fps=50)
fpth = "./gauss_siedelish_fixed_head.mp4"
ani.save(fpth, writer=writer)

### now explicit inefficient solution for no_flow_left boundary case

In [ ]:
h_imp2 = np.zeros_like(x)
G = Rx

LHS = np.zeros((nx,nx))
RHS = np.ones(nx) * G
for i in range(1,nx-1): 
    D = - hmean(k[i-1],k[i]) * b * dy / (dx)
    F = - hmean(k[i+1],k[i]) * b * dy / (dx)
    E = -D - F
    LHS[i,i-1] = D
    LHS[i,i]=E
    LHS[i,i+1]= F
    RHS[i] = G

LHS[0,0] = hmean(k[1],k[0]) * b * dy / (dx) 
LHS[0,1] = -hmean(k[1],k[0]) * b * dy / (dx) 
LHS[nx-1,nx-1] = 1

RHS[nx-1] = h0_xL

In [ ]:
h_imp2=np.linalg.solve(LHS,RHS)

In [ ]:
# initialize the heads 
h_old = np.mean((h0_x0,h0_xL)) * np.ones_like(x)
h_old[nx-1] = h0_xL

In [ ]:
conv=[np.inf]
h = np.atleast_2d(h_old.copy())
for j in range(10000):
    if conv[-1]>tol:
        h_curr = h[-1].copy()
        
        # left boundary first
        F = - hmean(k[1],k[0]) * b * dy / (dx)
        h_curr[0] = h_curr[1]-G/F
        
        for i in range(1, len(x)-1):
            D = - hmean(k[i-1],k[i]) * b * dy / (dx)
            F = - hmean(k[i+1],k[i]) * b * dy / (dx)
            E = -D - F
            h_curr[i] = (G-D*h_curr[i-1]-F*h_curr[i+1])/E
        h = np.vstack((h,h_curr))

        conv.append(np.sum(h[-1,:]-h_old)**2)
        h_old=h[-1,:].copy()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,4))
ax[0].plot(x,h_imp2, 'o-', label='"Truth"')
l0, = ax[0].plot([],[], label='Current head solution')
l1, = ax[1].plot([],[], lw=2)
line=[l0,l1]
ax[0].set_xlim((0,x.max()))
ax[1].set_xlim((0,x.max()))
ax[0].set_ylim((0,20))
ax[1].set_ylim((-.2,.2))
plt.suptitle('No Flow left Boundary')
def init():
    ax[0].set_ylim((0,20))
    ax[1].set_ylim((-.2,.2))
    ax[0].set_title('')
    ax[1].set_title('')


def update(i):
    if i==0:
        h_old = h[i,:]
    else:
        h_old = h[i-1,:]
    line[0].set_data(x, h[i,:])
    ax[0].legend()
    line[1].set_data(x, h[i,:]-h_old)
    ax[0].set_title(f'Head Solution: iter {i}')
    ax[1].set_title(f'Error = {conv[i]:.4e}')
    return line
ani = FuncAnimation(fig, update, range(h.shape[0]), init_func=init)
writer = FFMpegWriter(fps=50)
fpth = "./gauss_siedelish_no_flow_left.mp4"
ani.save(fpth, writer=writer)